<h1> SourceDataModel 

<h3> in database in tabel Sales_salesterritory attributt 'Group' veranderd naar 'Group1' vanwege error in code`

In [30]:
import pandas as pd
import sqlite3
import warnings
import pyodbc
import numpy as np
import os

warnings.simplefilter('ignore')
print(pyodbc.drivers())

['ODBC Driver 18 for SQL Server', 'ODBC Driver 17 for SQL Server']


<h3> inlezen database: AdventureWorks

In [31]:
# Path to the folder containing the CSV files
csv_folder = "databases/AdventureWorks"

# List all CSV files in the folder
csv_files = [f for f in os.listdir(csv_folder) if f.endswith('.csv')]
print(f"Found CSV files: {csv_files}")

# Initialize a dictionary to store DataFrames
df_AdventureWorks = {}

# Loop through each CSV file and load it into a DataFrame
for csv_file in csv_files:
    file_path = os.path.join(csv_folder, csv_file)
    print(f"Loading file: {file_path}")
    try:
        # Try reading the file with a fallback encoding
        df = pd.read_csv(file_path, encoding='latin1')  # Use 'latin1' or 'iso-8859-1' if UTF-8 fails
        # Store the DataFrame in the dictionary with the table name as the key
        table_name = os.path.splitext(csv_file)[0]
        df_AdventureWorks[table_name] = df
    except Exception as e:
        print(f"Failed to load {csv_file}: {e}")

# Access individual DataFrames by their table name
for table_name, df in df_AdventureWorks.items():
    print(f"Table: {table_name}, Rows: {len(df)}\n{df.head()}\n")
    print("Columns:", df.columns.tolist())


#function to clean nan values
def clean_nan_values(dw):
    for table_name, df in dw.items():
        # Zet alle NaN naar None zodat SQL Server NULL kan verwerken
        dw[table_name] = df.astype(object).where(pd.notnull(df), None)
    return dw

# Clean NaN values
df_AdventureWorks = clean_nan_values(df_AdventureWorks)




Found CSV files: ['Sales_Store.csv', 'Production_BillOfMaterials.csv', 'Sales_SalesTerritory.csv', 'Person_Address.csv', 'Purchasing_Vendor.csv', 'Person_Person.csv', 'Purchasing_PurchaseOrderHeader.csv', 'Sales_SalesOrderDetail.csv', 'HumanResources_Employee.csv', 'Production_ProductCategory.csv', 'Production_Product.csv', 'HumanResources_Department.csv', 'Sales_SalesOrderHeader.csv', 'Purchasing_PurchaseOrderDetail.csv', 'Sales_Customer.csv']
Loading file: databases/AdventureWorks/Sales_Store.csv
Loading file: databases/AdventureWorks/Production_BillOfMaterials.csv
Loading file: databases/AdventureWorks/Sales_SalesTerritory.csv
Loading file: databases/AdventureWorks/Person_Address.csv
Loading file: databases/AdventureWorks/Purchasing_Vendor.csv
Loading file: databases/AdventureWorks/Person_Person.csv
Loading file: databases/AdventureWorks/Purchasing_PurchaseOrderHeader.csv
Loading file: databases/AdventureWorks/Sales_SalesOrderDetail.csv
Loading file: databases/AdventureWorks/HumanRe

<h3> inlezen database: aenc.sqlite

In [32]:
# Path to the SQLite database
sqlite_file = "databases/aenc.sqlite"

# Connect to the SQLite database
conn = sqlite3.connect(sqlite_file)

# Get a list of all tables in the database
query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = pd.read_sql(query, conn)
table_names = tables['name'].tolist()
print(f"Found tables: {table_names}")

# Initialize a dictionary to store DataFrames
df_aenc = {}

# Loop through each table and load it into a DataFrame
for table_name in table_names:
    print(f"Loading table: {table_name}")
    try:
        # Read the table into a DataFrame
        df = pd.read_sql(f"SELECT * FROM {table_name};", conn)
        # Store the DataFrame in the dictionary with the table name as the key
        df_aenc[table_name] = df
    except Exception as e:
        print(f"Failed to load table {table_name}: {e}")

# Close the database connection
conn.close()

# Access individual DataFrames by their table name
for table_name, df in df_aenc.items():
    print(f"Table: {table_name}, Rows: {len(df)}\n{df.head()}\n")


# Clean NaN values in the DataFrames
df_aenc = clean_nan_values(df_aenc)

Found tables: ['Department', 'Product', 'State', 'Bonus', 'Customer', 'Employee', 'Sales_Order', 'Sales_Order_Item']
Loading table: Department
Loading table: Product
Loading table: State
Loading table: Bonus
Loading table: Customer
Loading table: Employee
Loading table: Sales_Order
Loading table: Sales_Order_Item
Table: Department, Rows: 5
   dept_id  dept_name  dept_head_id
0      100      R & D           501
1      200      Sales           902
2      300    Finance          1293
3      400  Marketing          1576
4      500   Shipping           703

Table: Product, Rows: 10
    id          name description          prod_size   color  quantity  \
0  300     Tee Shirt    Tank Top              Small   White        18   
1  301     Tee Shirt      V-neck             Medium  Orange        39   
2  302     Tee Shirt   Crew Neck  One size fits all   Black        72   
3  400  Baseball Cap  Cotton Cap  One size fits all   Black        92   
4  401  Baseball Cap    Wool cap  One size fits all

<h3> Inlezen database: NorthWind SQL server

In [33]:
# Verbindingsgegevens
server = '127.0.0.1'        
port = '1433'               
database = 'NorthWind'         
username = 'SA'             
password = 'iDTyjZx7dRL4'  

# Connection string
connection_string = (
    f"DRIVER={{ODBC Driver 17 for SQL Server}};"
    f"SERVER={server},{port};"
    f"DATABASE={database};"
    f"UID={username};"
    f"PWD={password};"
    "TrustServerCertificate=yes;"
    "Timeout=30;"
)

# Maak verbinding met de database
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

# Haal alle tabellen op
cursor.execute("SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE = 'BASE TABLE'")
tables = [row.TABLE_NAME for row in cursor.fetchall()]

# Dictionary om alle dataframes op te slaan
df_NorthWind = {}

# Loop door alle tabellen en laad ze in Pandas DataFrames
for table in tables:
    query = f"SELECT * FROM [{table}]"
    df = pd.read_sql(query, conn)
    df_NorthWind[table] = df
    print(f"Tabel '{table}' ingelezen met {df.shape[0]} rijen en {df.shape[1]} kolommen.")

# Sluit de verbinding
conn.close()

# Print de kolomnamen en de eerste paar rijen van elke DataFrame
for table_name, df in df_NorthWind.items():
    print(f"\nTable: {table_name}")
    print("Columns:", df.columns.tolist())
    print(df.head())
    




Tabel 'Employees' ingelezen met 9 rijen en 18 kolommen.
Tabel 'Categories' ingelezen met 8 rijen en 4 kolommen.
Tabel 'Customers' ingelezen met 91 rijen en 11 kolommen.
Tabel 'Shippers' ingelezen met 3 rijen en 3 kolommen.
Tabel 'Suppliers' ingelezen met 29 rijen en 12 kolommen.
Tabel 'Orders' ingelezen met 830 rijen en 14 kolommen.
Tabel 'Products' ingelezen met 77 rijen en 10 kolommen.
Tabel 'OrderDetails' ingelezen met 2155 rijen en 5 kolommen.
Tabel 'CustomerCustomerDemo' ingelezen met 0 rijen en 2 kolommen.
Tabel 'CustomerDemographics' ingelezen met 0 rijen en 2 kolommen.
Tabel 'Region' ingelezen met 4 rijen en 2 kolommen.
Tabel 'Territories' ingelezen met 53 rijen en 3 kolommen.
Tabel 'EmployeeTerritories' ingelezen met 49 rijen en 2 kolommen.

Table: Employees
Columns: ['EmployeeID', 'LastName', 'FirstName', 'Title', 'TitleOfCourtesy', 'BirthDate', 'HireDate', 'Address', 'City', 'Region', 'PostalCode', 'Country', 'HomePhone', 'Extension', 'Photo', 'Notes', 'ReportsTo', 'PhotoPat

<h3>maak dictionary aan voor alle goede dataframes + mapping dictionary


In [34]:
dfs_sourcedatamodel = {}
df_mapping = {}

<h3> table: Production_ProductCategory
<h4> bronnen: Production_ProductCategory + Categories

In [35]:
# Data opnieuw laden om dubbele kolommen te voorkomen
df_product_category = df_AdventureWorks.get("Production_ProductCategory").copy()
df_categories = df_NorthWind.get("Categories").copy()

# Voeg 'Source' kolom toe
df_product_category['Source'] = 'AdventureWorks'
df_categories['Source'] = 'NorthWind'


# Voeg een unieke sleutel toe (MergedCategoryID) als deze nog niet bestaat
if "MergedCategoryID" not in df_product_category.columns:
    df_product_category.insert(0, "MergedCategoryID", range(1, len(df_product_category) + 1))

if "MergedCategoryID" not in df_categories.columns:
    df_categories.insert(0, "MergedCategoryID", range(len(df_product_category) + 1, len(df_product_category) + len(df_categories) + 1))

# Hernoem kolommen zodat ze overeenkomen
df_categories.rename(columns={
    "CategoryID": "ProductCategoryID", 
    "CategoryName": "Name"
}, inplace=True)

# Combineer de twee tabellen onder elkaar
merged_df_productCategorie = pd.concat([df_product_category, df_categories], ignore_index=True)

# Vul NaN-waarden in met lege strings (optioneel)
merged_df_productCategorie.fillna("", inplace=True)

# Verwijder de kolom ShipDate als deze bestaat
if "Picture" in merged_df_productCategorie.columns:
    merged_df_productCategorie = merged_df_productCategorie.drop(columns=["Picture"])



#mapping van de categorieen per bron
category_mapping = merged_df_productCategorie[['ProductCategoryID', 'Source', 'MergedCategoryID']]

dfs_sourcedatamodel["Production_ProductCategory"] = merged_df_productCategorie
df_mapping["category_mapping"] = category_mapping



# Print het resultaat
print(merged_df_productCategorie)



    MergedCategoryID ProductCategoryID            Name  \
0                  1                 1           Bikes   
1                  2                 2      Components   
2                  3                 3        Clothing   
3                  4                 4     Accessories   
4                  5                 1       Beverages   
5                  6                 2      Condiments   
6                  7                 3     Confections   
7                  8                 4  Dairy Products   
8                  9                 5  Grains/Cereals   
9                 10                 6    Meat/Poultry   
10                11                 7         Produce   
11                12                 8         Seafood   

                                  rowguid       ModifiedDate          Source  \
0   {CFBDA25C-DF71-47A7-B81B64EE161AA37C}  04/30/18 00:00:00  AdventureWorks   
1   {C657828D-D808-4ABA-91A3AF2CE02300E9}  04/30/18 00:00:00  AdventureWorks   
2   {

<h3> table: Production_Product
<h4> bronnen: Production_Product + Products + Product

In [36]:
# Functie om alle datums correct te converteren
def convert_dates(df):
    date_columns = [col for col in df.columns if "date" in col.lower()]
    for col in date_columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')  # Zet om naar datetime
    return df

# Functie om -1 te vervangen door None
def clean_negative_values(df):
    for col in df.select_dtypes(include=[np.number]).columns:
        df[col] = df[col].apply(lambda x: None if x == -1 else x)
    return df

# Data opnieuw laden om dubbele kolommen te voorkomen
df_production_product = df_AdventureWorks.get("Production_Product").copy()
df_products = df_NorthWind.get("Products").copy()
df_product = df_aenc.get("Product").copy()

# Voeg 'Source' kolom toe
df_production_product['Source'] = 'AdventureWorks'
df_products['Source'] = 'NorthWind'
df_product['Source'] = 'AENC'

# Maak een unieke MergedID voor elke rij
df_production_product.insert(0, "MergedID", range(1, len(df_production_product) + 1))
df_products.insert(0, "MergedID", range(len(df_production_product) + 1, len(df_production_product) + len(df_products) + 1))
df_product.insert(0, "MergedID", range(len(df_production_product) + len(df_products) + 1, 
                                       len(df_production_product) + len(df_products) + len(df_product) + 1))

# Zorg dat alle tabellen dezelfde kolommen hebben
df_products.rename(columns={"ProductName": "Name", "CategoryID": "ProductCategoryID", "UnitPrice": "ListPrice"}, inplace=True)
df_product.rename(columns={"id": "ProductID", "name": "Name", "prod_size": "Size", "unit_price": "ListPrice", "Category": "ProductCategoryID"}, inplace=True)

# Voeg ontbrekende kolommen toe met lege waarden
all_columns = set(df_production_product.columns).union(set(df_products.columns)).union(set(df_product.columns))
for df in [df_production_product, df_products, df_product]:
    for col in all_columns:
        if col not in df.columns:
            df[col] = None

# Zet alle tabellen onder elkaar
merged_df_product = pd.concat([df_production_product, df_products, df_product], ignore_index=True)

# Datumconversie toepassen
merged_df_product = convert_dates(merged_df_product)

# Negatieve waarden verwijderen
merged_df_product = clean_negative_values(merged_df_product)

# Verwijder 'picture_name' en 'DiscontinuedDate' kolommen
merged_df_product.drop(columns=["picture_name"], inplace=True, errors="ignore")

# Hernoem 'ReorderLevel' naar 'ReorderPoint' en zet ze samen in één kolom
if "ReorderLevel" in merged_df_product.columns and "ReorderPoint" in merged_df_product.columns:
    merged_df_product["ReorderPoint"] = merged_df_product["ReorderPoint"].combine_first(merged_df_product["ReorderLevel"])
    merged_df_product.drop(columns=["ReorderLevel"], inplace=True)

# Hernoem 'color' zodat beide versies samengevoegd worden in één kolom
if "color" in merged_df_product.columns and "Color" in merged_df_product.columns:
    merged_df_product["Color"] = merged_df_product["Color"].combine_first(merged_df_product["color"])
    merged_df_product.drop(columns=["color"], inplace=True)

product_mapping = merged_df_product[['ProductID', 'Source', 'MergedID']]

# Merge op ProductCategoryID + Source om conflicten te vermijden
merged_df_product = merged_df_product.merge(
    category_mapping,
    how='left',
    left_on=['ProductCategoryID', 'Source'],
    right_on=['ProductCategoryID', 'Source']
)

# Vervang de oude ProductCategoryID door de nieuwe MergedCategoryID
merged_df_product['ProductCategoryID'] = merged_df_product['MergedCategoryID']
merged_df_product.drop(columns=['MergedCategoryID'], inplace=True, errors='ignore')

# Verwijder de kolom ShipDate als deze bestaat
if "DiscontinuedDate" in merged_df_product.columns:
    merged_df_product = merged_df_product.drop(columns=["DiscontinuedDate"])

# Data opslaan in het data warehouse dictionary
dfs_sourcedatamodel["Production_Product"] = merged_df_product
df_mapping["product_mapping"] = product_mapping

# Print het resultaat
print(merged_df_product)

     MergedID ProductID                   Name ProductNumber MakeFlag  \
0           1         1        Adjustable Race       AR-5381        0   
1           2         2           Bearing Ball       BA-8327        0   
2           3         3        BB Ball Bearing       BE-2349       -1   
3           4         4  Headset Ball Bearings       BE-2908        0   
4           5       316                  Blade       BL-2036       -1   
..        ...       ...                    ...           ...      ...   
586       587       500                  Visor          None     None   
587       588       501                  Visor          None     None   
588       589       600             Sweatshirt          None     None   
589       590       601             Sweatshirt          None     None   
590       591       700                 Shorts          None     None   

    FinishedGoodsFlag  Color SafetyStockLevel ReorderPoint StandardCost  ...  \
0                   0   None             10

<h3> table: Purchasing_Vendor

In [37]:

#get dataframe from dictionary
purchasing_vendor_df = df_AdventureWorks.get("Purchasing_Vendor")

# Check if the DataFrame exists
if purchasing_vendor_df is not None:
    # Add the DataFrame to the sourcedatamodel dictionary
    dfs_sourcedatamodel["Purchasing_Vendor"] = purchasing_vendor_df

    print(f"Rows: {len(purchasing_vendor_df)}")
    print(purchasing_vendor_df.head())
else:
    print("The table 'Purchasing_Vendor' does not exist in database.")

Rows: 104
  BusinessEntityID AccountNumber                     Name CreditRating  \
0             1492  AUSTRALI0001  Australia Bike Retailer            1   
1             1494  ALLENSON0001          Allenson Cycles            2   
2             1496  ADVANCED0001        Advanced Bicycles            1   
3             1498    TRIKES0001             Trikes, Inc.            2   
4             1500   MORGANB0001  Morgan Bike Accessories            1   

  PreferredVendorStatus ActiveFlag PurchasingWebServiceURL       ModifiedDate  
0                    -1         -1                    None  12/23/21 00:00:00  
1                    -1         -1                    None  04/25/21 00:00:00  
2                    -1         -1                    None  04/25/21 00:00:00  
3                    -1         -1                    None  02/03/22 00:00:00  
4                    -1         -1                    None  02/02/22 00:00:00  


<h3> table: sales_store

In [38]:

#get dataframe from dictionary
sales_store_df = df_AdventureWorks.get("Sales_Store")

# Check if the DataFrame exists
if sales_store_df is not None:
    # Add the DataFrame to the sourcedatamodel dictionary
    dfs_sourcedatamodel["Sales_Store"] = sales_store_df

    print(f"Rows: {len(sales_store_df)}")
    print(sales_store_df.head())
else:
    print("The table does not exist in database.")



Rows: 701
  BusinessEntityID                            Name SalesPersonID  \
0              292            Next-Door Bike Store           279   
1              294  Professional Sales and Service           276   
2              296                  Riders Company           277   
3              298              The Bike Mechanics           275   
4              300               Nationwide Supply           286   

                                 rowguid       ModifiedDate  
0  {A22517E3-848D-4EBE-B9D97437F3432304}  09/12/14 11:15:07  
1  {B50CA50B-C601-4A13-B07E2C63862D71B4}  09/12/14 11:15:07  
2  {337C3688-1339-4E1A-A08AB54B23566E49}  09/12/14 11:15:07  
3  {7894F278-F0C8-4D16-BD75213FDBF13023}  09/12/14 11:15:07  
4  {C3FC9705-A8C4-4F3A-9550EB2FA4B7B64D}  09/12/14 11:15:07  


<h3> tabel: employee_territories

In [39]:

#get dataframe from dictionary
employeeTerritories_df = df_NorthWind.get("EmployeeTerritories")

# Check if the DataFrame exists
if employeeTerritories_df is not None:
     # Apply the same offset as Employee_df2 for AENC employees
    employeeTerritories_df["EmployeeID"] = employeeTerritories_df["EmployeeID"].astype(int) + 100000  
    # Add the DataFrame to the sourcedatamodel dictionary
    dfs_sourcedatamodel["EmployeeTerritories"] = employeeTerritories_df

    print(f" Rows: {len(employeeTerritories_df)}")
    print(employeeTerritories_df.head())
else:
    print("The table does not exist in database.")

 Rows: 49
   EmployeeID TerritoryID
0      100001       06897
1      100001       19713
2      100002       01581
3      100002       01730
4      100002       01833


<h3> table: Territories

In [40]:
#get dataframe from dictionary
territories_df = df_NorthWind.get("Territories")

# Check if the DataFrame exists
if territories_df is not None:
    # Add the DataFrame to the sourcedatamodel dictionary
    dfs_sourcedatamodel["Territories"] = territories_df

    print(f" Rows: {len(territories_df)}")
    print(territories_df.head())
else:
    print("The table does not exist in database.")

 Rows: 53
  TerritoryID                               TerritoryDescription  RegionID
0       01581  Westboro                                      ...         1
1       01730  Bedford                                       ...         1
2       01833  Georgetow                                     ...         1
3       02116  Boston                                        ...         1
4       02139  Cambridge                                     ...         1


<h3> table: Region

In [41]:
#get dataframe from dictionary
region_df = df_NorthWind.get("Region")

# Check if the DataFrame exists
if region_df is not None:
    # Add the DataFrame to the sourcedatamodel dictionary
    dfs_sourcedatamodel["Region"] = region_df

    print(f" Rows: {len(region_df)}")
    print(region_df.head())
else:
    print("The table does not exist in database.")

 Rows: 4
   RegionID                                  RegionDescription
0         1  Eastern                                       ...
1         2  Western                                       ...
2         3  Northern                                      ...
3         4  Southern                                      ...


<h3> Table: Bonus

In [42]:
# Get dataframe from dictionary
bonus_df = df_aenc.get("Bonus")
bonus_df = bonus_df.drop_duplicates(subset=["emp_id", "bonus_date"])


# Check if the DataFrame exists
if bonus_df is not None:
    # Apply the same offset as Employee_df2 for AENC employees
    bonus_df["emp_id"] = bonus_df["emp_id"].astype(int) + 200000  

    # Add the DataFrame to the sourcedatamodel dictionary
    dfs_sourcedatamodel["Bonus"] = bonus_df

    print(f"Rows: {len(bonus_df)}")
    print(bonus_df.head())
else:
    print("The table does not exist in database.")


Rows: 33
   emp_id bonus_date bonus_amount
0  200129  31-dec-24           50
1  200195  31-dec-23           50
2  200299  31-dec-23          200
3  201142  31-dec-23          100
4  200129  31-dec-22           50


<h3> table: Sales_Customer

<h4> Bronnen: Customer + Customers + Sales_Customer

In [43]:
#get dataframe from dictionary
Sales_Customer_df = df_AdventureWorks.get("Sales_Customer")


# Laden van de datasets (vervang dit met je eigen DataFrame-imports)
Sales_Customer_df = df_AdventureWorks.get("Sales_Customer")
customer_df = df_aenc.get("Customer")   
customers_df = df_NorthWind.get("Customers")
# Voeg 'Source' kolom toe
Sales_Customer_df["Source"] = "AdventureWorks"
customer_df["Source"] = "AENC"
customers_df["Source"] = "NorthWind"

# Zorg dat CustomerID overal een string is om inconsistenties te voorkomen
Sales_Customer_df["CustomerID"] = Sales_Customer_df["CustomerID"].astype(str)
customers_df["CustomerID"] = customers_df["CustomerID"].astype(str)
customer_df.rename(columns={"id": "CustomerID"}, inplace=True)
customer_df["CustomerID"] = customer_df["CustomerID"].astype(str)

# Hernoem kolommen om aan te sluiten bij Sales_Customer structuur
customers_df.rename(columns={"CompanyName": "CompanyName"}, inplace=True)
customer_df.rename(columns={"company_name": "CompanyName"}, inplace=True)

# Houd alleen de relevante kolommen
Sales_Customer_df = Sales_Customer_df[["CustomerID", "PersonID", "StoreID", "TerritoryID", "AccountNumber", "rowguid", "ModifiedDate", "Source"]]
customer_df = customer_df[["CustomerID", "CompanyName", "Source"]]
customers_df = customers_df[["CustomerID", "CompanyName", "Source"]]

# Voeg een unieke sleutel toe (MergedCustomerID)
Sales_Customer_df.insert(0, "MergedCustomerID", range(1, len(Sales_Customer_df) + 1))
customer_df.insert(0, "MergedCustomerID", range(len(Sales_Customer_df) + 1, len(Sales_Customer_df) + len(customer_df) + 1))
customers_df.insert(0, "MergedCustomerID", range(len(Sales_Customer_df) + len(customer_df) + 1, len(Sales_Customer_df) + len(customer_df) + len(customers_df) + 1))

# Combineer de tabellen onder elkaar
merged_customers = pd.concat([Sales_Customer_df, customer_df, customers_df], ignore_index=True)

# Vul NaN-waarden in met None (SQL-compatible)
merged_customers = merged_customers.where(pd.notna(merged_customers), None)

# Mapping maken voor toekomstige foreign key updates
customer_mapping = merged_customers[["CustomerID", "Source", "MergedCustomerID"]]



dfs_sourcedatamodel["Sales_Customer"] = merged_customers
df_mapping["customer_mapping"] = customer_mapping



# Print het resultaat
print(merged_customers)


       MergedCustomerID CustomerID PersonID StoreID TerritoryID AccountNumber  \
0                     1          1     None   934.0           1    AW00000001   
1                     2          2     None  1028.0           1    AW00000002   
2                     3          3     None   642.0           4    AW00000003   
3                     4          4     None   932.0           4    AW00000004   
4                     5          5     None  1026.0           4    AW00000005   
...                 ...        ...      ...     ...         ...           ...   
20032             20033      WARTH     None    None        None          None   
20033             20034      WELLI     None    None        None          None   
20034             20035      WHITC     None    None        None          None   
20035             20036      WILMK     None    None        None          None   
20036             20037      WOLZA     None    None        None          None   

                           

<h3> table: Sales_SalesTerritory

In [44]:
#get dataframe from dictionary
Sales_SalesTerritory_df = df_AdventureWorks.get("Sales_SalesTerritory")
Sales_SalesTerritory_df.rename(columns={"Group": "Group1"}, inplace=True)

# Check if the DataFrame exists
if Sales_SalesTerritory_df is not None:

    # Add the DataFrame to the sourcedatamodel dictionary
    dfs_sourcedatamodel["Sales_SalesTerritory"] = Sales_SalesTerritory_df

    print(f" Rows: {len(Sales_SalesTerritory_df)}")
    print(Sales_SalesTerritory_df.head())
else:
    print("The table does not exist in database.")




 Rows: 10
  TerritoryID       Name CountryRegionCode         Group1       SalesYTD  \
0           1  Northwest                US  North America   7887186.7882   
1           2  Northeast                US  North America   2402176.8476   
2           3    Central                US  North America    3072175.118   
3           4  Southwest                US  North America  10510853.8739   
4           5  Southeast                US  North America   2538667.2515   

  SalesLastYear CostYTD CostLastYear                                rowguid  \
0  3298694.4938     0.0          0.0  {43689A10-E30B-497F-B0DE11DE20267FF7}   
1  3607148.9371     0.0          0.0  {00FB7309-96CC-49E2-83630A1BA72486F2}   
2  3205014.0767     0.0          0.0  {DF6E7FD8-1A8D-468C-B103ED8ADDB452C1}   
3  5366575.7098     0.0          0.0  {DC3E9EA0-7950-4431-942899DBCBC33865}   
4  3925071.4318     0.0          0.0  {6DC4165A-5E4C-42D2-809D4344E0AC75E7}   

        ModifiedDate  
0  04/30/08 00:00:00  
1  04/30/08 

<h3> table: Purchasing_PurchaseOrderHeader


In [45]:

#get dataframe from dictionary
Purchasing_PurchaseOrderHeader = df_AdventureWorks.get("Purchasing_PurchaseOrderHeader")

# Check if the DataFrame exists
if Purchasing_PurchaseOrderHeader is not None:
    # Add the DataFrame to the sourcedatamodel dictionary
    dfs_sourcedatamodel["Purchasing_PurchaseOrderHeader"] = Purchasing_PurchaseOrderHeader

    print(f" Rows: {len(Purchasing_PurchaseOrderHeader)}")
    print(Purchasing_PurchaseOrderHeader.head())
else:
    print("The table does not exist in database.")

 Rows: 4012
  PurchaseOrderID RevisionNumber Status EmployeeID VendorID ShipMethodID  \
0               1              4      4        258     1580            3   
1               2              4      1        254     1496            5   
2               3              4      4        257     1494            2   
3               4              4      3        261     1650            5   
4               5              4      4        251     1654            4   

           OrderDate           ShipDate  SubTotal    TaxAmt   Freight  \
0  04/16/21 00:00:00  04/25/21 00:00:00    201.04   16.0832     5.026   
1  04/16/21 00:00:00  04/25/21 00:00:00  272.1015   21.7681    6.8025   
2  04/16/21 00:00:00  04/25/21 00:00:00    8847.3   707.784  221.1825   
3  04/16/21 00:00:00  04/25/21 00:00:00  171.0765   13.6861    4.2769   
4  04/30/21 00:00:00  05/09/21 00:00:00   20397.3  1631.784  509.9325   

     TotalDue       ModifiedDate  
0    222.1492  04/25/21 00:00:00  
1    300.6721  04/25/2

<h3> table: Purchasing_PurchaseOrderDetail

In [46]:
# Get the Purchasing_PurchaseOrderDetail DataFrame from the dictionary
Purchasing_PurchaseOrderDetail = df_AdventureWorks.get("Purchasing_PurchaseOrderDetail")

# Check if the DataFrame exists
if Purchasing_PurchaseOrderDetail is not None:
    # Add the DataFrame to the sourcedatamodel dictionary
    dfs_sourcedatamodel["Purchasing_PurchaseOrderDetail"] = Purchasing_PurchaseOrderDetail

    # Perform the merge to replace ProductID with MergedID
    Purchasing_PurchaseOrderDetail = Purchasing_PurchaseOrderDetail.merge(
        product_mapping,  # This contains ProductID, Source, and MergedID
        how='left',
        left_on=['ProductID'],  # Assuming ProductID exists in this table
        right_on=['ProductID']
    )

    # Replace ProductID with MergedID
    Purchasing_PurchaseOrderDetail['ProductID'] = Purchasing_PurchaseOrderDetail['MergedID']

    # Drop the MergedID column as it's no longer needed
    Purchasing_PurchaseOrderDetail.drop(columns=['MergedID'], inplace=True, errors='ignore')

    # Drop rows where Source is 'NorthWind'
    if 'Source' in Purchasing_PurchaseOrderDetail.columns:
        Purchasing_PurchaseOrderDetail = Purchasing_PurchaseOrderDetail[Purchasing_PurchaseOrderDetail['Source'] != 'NorthWind']
        Purchasing_PurchaseOrderDetail.reset_index(drop=True, inplace=True)
        print("Rows with Source == 'NorthWind' have been removed.")
    else:
        print("The 'Source' column does not exist in Purchasing_PurchaseOrderDetail.")

    # Update the dictionary with the modified DataFrame
    dfs_sourcedatamodel["Purchasing_PurchaseOrderDetail"] = Purchasing_PurchaseOrderDetail

    # Print the result
    print(f"Rows: {len(Purchasing_PurchaseOrderDetail)}")
    print(Purchasing_PurchaseOrderDetail.head())

   
    print("The updated table has been saved to 'Purchasing_PurchaseOrderDetail.csv'.")
else:
    print("The table 'Purchasing_PurchaseOrderDetail' does not exist in the database.")

Rows with Source == 'NorthWind' have been removed.
Rows: 8845
  PurchaseOrderID PurchaseOrderDetailID            DueDate OrderQty  \
0               1                     1  04/30/21 00:00:00        4   
1               2                     2  04/30/21 00:00:00        3   
2               2                     3  04/30/21 00:00:00        3   
3               3                     4  04/30/21 00:00:00      550   
4               4                     5  04/30/21 00:00:00        3   

   ProductID UnitPrice LineTotal ReceivedQty RejectedQty StockedQty  \
0          1     50.26    201.04         3.0         0.0        3.0   
1         38     45.12    135.36         3.0         0.0        3.0   
2         39   45.5805  136.7415         3.0         0.0        3.0   
3        203    16.086    8847.3       550.0         0.0      550.0   
4          4   57.0255  171.0765         2.0         1.0        1.0   

        ModifiedDate          Source  
0  04/23/21 00:00:00  AdventureWorks  
1  04/

<h3> table: Suppliers

In [47]:
#get dataframe from dictionary
Suppliers = df_NorthWind.get("Suppliers")


# Check if the DataFrame exists
if Suppliers is not None:
    # Add the DataFrame to the sourcedatamodel dictionary
    dfs_sourcedatamodel["Suppliers"] = Suppliers

    print(f" Rows: {len(Suppliers)}")
    print(Suppliers.head())
else:
    print("The table does not exist in database.")

 Rows: 29
   SupplierID                         CompanyName                 ContactName  \
0           1                      Exotic Liquids            Charlotte Cooper   
1           2          New Orleans Cajun Delights               Shelley Burke   
2           3           Grandma Kelly's Homestead               Regina Murphy   
3           4                       Tokyo Traders                Yoshi Nagase   
4           5  Cooperativa de Quesos 'Las Cabras'  Antonio del Valle Saavedra   

           ContactTitle                    Address         City    Region  \
0    Purchasing Manager             49 Gilbert St.       London      None   
1   Order Administrator             P.O. Box 78934  New Orleans        LA   
2  Sales Representative             707 Oxford Rd.    Ann Arbor        MI   
3     Marketing Manager  9-8 Sekimai Musashino-shi        Tokyo      None   
4  Export Administrator          Calle del Rosal 4       Oviedo  Asturias   

  PostalCode Country           Phone    

<h3> table: Person_Person
<h4> Bronnen: Person_Person + Employees + Employee

In [48]:

# Laden van de datasets (Vervang dit met je eigen DataFrame-imports)
Person_Person_df = df_AdventureWorks.get("Person_Person")
Employees_df = df_NorthWind.get("Employees")
Employee_df = df_aenc.get("Employee")

# Voeg 'Source' kolom toe om de herkomst te behouden
Person_Person_df["Source"] = "AdventureWorks"
Employees_df["Source"] = "Northwind"
Employee_df["Source"] = "aenc"

# Hernoem kolommen zodat ze consistent zijn met Person_Person
Employees_df.rename(columns={"EmployeeID": "BusinessEntityID", "HomePhone": "PhoneNumber"}, inplace=True)
Employee_df.rename(columns={"emp_id": "BusinessEntityID", "phone": "PhoneNumber"}, inplace=True)

# Voeg een unieke sleutel toe (MergedBusinessEntityID) om overlaps te voorkomen
Person_Person_df["MergedBusinessEntityID"] = Person_Person_df["BusinessEntityID"]
Employees_df["MergedBusinessEntityID"] = Employees_df["BusinessEntityID"] + 100000  # Offset voor Northwind
Employee_df["MergedBusinessEntityID"] = Employee_df["BusinessEntityID"] + 200000  # Offset voor AENC

# Houd alleen de relevante kolommen
Person_Person_df = Person_Person_df[["MergedBusinessEntityID", "BusinessEntityID", "PersonType", "NameStyle", "Title", "LastName", "Suffix", "EmailPromotion", "rowguid",  "ModifiedDate", "Source"]]
Employees_df = Employees_df[["MergedBusinessEntityID", "BusinessEntityID", "Title", "LastName", "PhoneNumber", "Source"]]
Employee_df = Employee_df[["MergedBusinessEntityID", "BusinessEntityID", "emp_fname", "emp_lname", "PhoneNumber", "Source"]]

# Hernoemen van kolommen zodat ze overeenkomen
Employees_df.rename(columns={"Title": "Title", "LastName": "LastName"}, inplace=True)
Employee_df.rename(columns={"emp_fname": "Title", "emp_lname": "LastName"}, inplace=True)

# Samenvoegen van de tabellen zonder merge (alle data onder elkaar)
merged_person = pd.concat([Person_Person_df, Employees_df, Employee_df], ignore_index=True)

dfs_sourcedatamodel["Person_Person"] = merged_person


# Mapping maken voor toekomstige foreign key updates
person_mapping = merged_person[["BusinessEntityID", "Source", "MergedBusinessEntityID"]]
df_mapping["person_mapping"] = person_mapping

# Print het resultaat
print(merged_person)


      MergedBusinessEntityID BusinessEntityID PersonType NameStyle    Title  \
0                          1                1         EM         0     None   
1                          2                2         EM         0     None   
2                          3                3         EM         0     None   
3                          4                4         EM         0     None   
4                          5                5         EM         0      Ms.   
...                      ...              ...        ...       ...      ...   
20051                 200921              921        NaN       NaN  Charles   
20052                 200930              930        NaN       NaN      Ann   
20053                 200949              949        NaN       NaN   Pamela   
20054                 200958              958        NaN       NaN   Thomas   
20055                 200992              992        NaN       NaN    Joyce   

          LastName Suffix EmailPromotion  \
0      

<h3> table: HumanResources_Department
<h4> bronnen: HumanResources_Department + Department

In [49]:

# Laden van de datasets (vervang met echte DataFrame-imports)
HumanResources_Department_df = df_AdventureWorks.get("HumanResources_Department")
Department_df = df_aenc.get("Department")

# Voeg 'Source' kolom toe
HumanResources_Department_df["Source"] = "AdventureWorks"
Department_df["Source"] = "AENC"

# Hernoem kolommen zodat ze consistent zijn
Department_df.rename(columns={"dept_id": "DepartmentID", "dept_name": "Name"}, inplace=True)

# Houd alleen de relevante kolommen
HumanResources_Department_df = HumanResources_Department_df[["DepartmentID", "Name", "GroupName", "ModifiedDate", "Source"]]
Department_df = Department_df[["DepartmentID", "Name", "Source"]]

# Voeg een placeholder toe voor ontbrekende GroupName en ModifiedDate in AENC-data
Department_df["GroupName"] = None
Department_df["ModifiedDate"] = None

# Samenvoegen van de tabellen zonder merge (alle data onder elkaar)
merged_departments = pd.concat([HumanResources_Department_df, Department_df], ignore_index=True)

dfs_sourcedatamodel["HumanResources_Department"] = merged_departments

print(HumanResources_Department_df["GroupName"].str.len().max())  # Check maximale lengte 

# Print het resultaat
print(merged_departments)


36
   DepartmentID                        Name  \
0             1                 Engineering   
1             2                 Tool Design   
2             3                       Sales   
3             4                   Marketing   
4             5                  Purchasing   
5             6    Research and Development   
6             7                  Production   
7             8          Production Control   
8             9             Human Resources   
9            10                     Finance   
10           11        Information Services   
11           12            Document Control   
12           13           Quality Assurance   
13           14  Facilities and Maintenance   
14           15      Shipping and Receiving   
15           16                   Executive   
16          100                       R & D   
17          200                       Sales   
18          300                     Finance   
19          400                   Marketing   
20        

<h3> table: HumanResources_Employee
<h4> Bronnen: HumanResources_Employee + Employees

In [50]:

# Laden van de datasets (Vervang dit met je eigen DataFrame-imports)
Person_Person_df = df_AdventureWorks.get("Person_Person")
Employees_df2 = df_NorthWind.get("Employees")
Employee_df2 = df_aenc.get("Employee")
HumanResources_Employee_df = df_AdventureWorks.get("HumanResources_Employee")  # Nieuwe tabel

# Voeg 'Source' kolom toe om de herkomst te behouden
Person_Person_df["Source"] = "AdventureWorks"
Employees_df2["Source"] = "NorthWind"
Employee_df2["Source"] = "AENC"

# Hernoem kolommen zodat ze consistent zijn met Person_Person
Employees_df2.rename(columns={"EmployeeID": "BusinessEntityID", "Title": "JobTitle", "BirthDate": "BirthDate", "HireDate": "HireDate", "TitleOfCourtesy": "Gender"}, inplace=True)
Employee_df2.rename(columns={"emp_id": "BusinessEntityID", "start_date": "HireDate", "birth_date": "BirthDate", "sex": "Gender", "salary": "salary", "manager_id": "ManagerID","dept_id":"DepartmentID"}, inplace=True)

# Voeg een unieke sleutel toe (MergedBusinessEntityID) om overlaps te voorkomen
Person_Person_df["MergedBusinessEntityID"] = Person_Person_df["BusinessEntityID"]
Employees_df2["MergedBusinessEntityID"] = Employees_df2["BusinessEntityID"] + 100000  # Offset voor Northwind
Employee_df2["MergedBusinessEntityID"] = Employee_df2["BusinessEntityID"] + 200000  # Offset voor AENC

# Voeg lege waarden toe voor ontbrekende kolommen
Employee_df2["JobTitle"] = None

# Mapping maken voor ManagerID verwijzing
manager_mapping = Employee_df2[["BusinessEntityID", "MergedBusinessEntityID"]].copy()
manager_mapping.rename(columns={"BusinessEntityID": "OldManagerID", "MergedBusinessEntityID": "NewManagerID"}, inplace=True)

# Houd alleen de relevante kolommen
Person_Person_df = Person_Person_df[["MergedBusinessEntityID", "BusinessEntityID", "Source"]]
Employees_df2 = Employees_df2[["MergedBusinessEntityID", "BusinessEntityID", "JobTitle", "BirthDate", "HireDate", "Gender", "Source"]]
Employee_df2 = Employee_df2[["MergedBusinessEntityID", "BusinessEntityID", "JobTitle", "BirthDate", "HireDate", "Gender", "salary", "ManagerID", "Source", "DepartmentID"]]

# Merge managerID correct op basis van de mapping
Employee_df2 = Employee_df2.merge(manager_mapping, how="left", left_on="ManagerID", right_on="OldManagerID")
Employee_df2.drop(columns=["OldManagerID", "ManagerID"], inplace=True)
Employee_df2.rename(columns={"NewManagerID": "ManagerID"}, inplace=True)

# Merge Person_Person met HumanResources_Employee op BusinessEntityID
Person_Person_df = Person_Person_df.merge(
    HumanResources_Employee_df[["BusinessEntityID", "DepartmentID", "BirthDate", "HireDate", "JobTitle", "Gender"]],
    on="BusinessEntityID",
    how="left"
)

# Samenvoegen van de tabellen zonder concat (één enkele tabel)
merged_human_resources_Employee = pd.concat([Person_Person_df, Employees_df2, Employee_df2], ignore_index=True)

# Pas de volgorde van de kolommen aan
column_order = ["MergedBusinessEntityID", "BusinessEntityID", "JobTitle", "BirthDate", "HireDate", "Gender", "salary", "ManagerID", "DepartmentID", "Source"]
merged_human_resources_Employee = merged_human_resources_Employee[column_order]

dfs_sourcedatamodel["HumanResources_Employee"] = merged_human_resources_Employee

# Mapping maken voor toekomstige foreign key updates
employee_mapping = merged_human_resources_Employee[["BusinessEntityID", "Source", "MergedBusinessEntityID"]]
df_mapping["employee_mapping"] = employee_mapping

# Print het resultaat
print(merged_human_resources_Employee)

merged_human_resources_Employee["salary"] = merged_human_resources_Employee["salary"].fillna(0).astype(float)




      MergedBusinessEntityID BusinessEntityID                       JobTitle  \
0                          1                1        Chief Executive Officer   
1                          2                2  Vice President of Engineering   
2                          3                3            Engineering Manager   
3                          4                4           Senior Tool Designer   
4                          5                5                Design Engineer   
...                      ...              ...                            ...   
20051                 200921              921                           None   
20052                 200930              930                           None   
20053                 200949              949                           None   
20054                 200958              958                           None   
20055                 200992              992                           None   

         BirthDate     HireDate Gender 

<h3> Table: Person_Address

In [51]:
#get dataframe from dictionary
Person_Adress_Df = df_AdventureWorks.get("Person_Address")
column_order = ["AddressID", "AddressLine1", "AddressLine2", "City", "StateProvinceID", "PostalCode","BusinessEntityID"]
Person_Adress_Df = Person_Adress_Df[column_order]

# Check if the DataFrame exists
if Person_Adress_Df is not None:
    # Add the DataFrame to the sourcedatamodel dictionary
    dfs_sourcedatamodel["Person_Address"] = Person_Adress_Df

    print(f"Rows: {len(Person_Adress_Df)}")
    print(Person_Adress_Df.head())
else:
    print("The table does not exist in database.")

Rows: 19614
  AddressID          AddressLine1 AddressLine2     City StateProvinceID  \
0         1         1970 Napa Ct.         None  Bothell              79   
1         2    9833 Mt. Dias Blv.         None  Bothell              79   
2         3  7484 Roundtree Drive         None  Bothell              79   
3         4      9539 Glenside Dr         None  Bothell              79   
4         5         1226 Shoe St.         None  Bothell              79   

  PostalCode BusinessEntityID  
0      98011               12  
1      98011              123  
2      98011              285  
3      98011              251  
4      98011              124  


<h3> table: Shippers
<h4> bronnen: Shippers + Orders 

In [52]:
#get dataframe from dictionary
Shippers = df_NorthWind.get("Shippers")

# Check if the DataFrame exists
if Shippers is not None:
    # Add the DataFrame to the sourcedatamodel dictionary
    dfs_sourcedatamodel["Shippers"] = Shippers

    print(f" Rows: {len(Shippers)}")
    print(Shippers.head())
else:
    print("The table does not exist in database.")

 Rows: 3
   ShipperID       CompanyName           Phone
0          1    Speedy Express  (503) 555-9831
1          2    United Package  (503) 555-3199
2          3  Federal Shipping  (503) 555-9931


<h3> table: Sales_SalesOrderHeader
<h4> Bronnen: Sales_SalesOrderHeader + Orders + Sales_Order

In [53]:

#  1. Laad de datasets 
sales_adventureworks_df = df_AdventureWorks.get("Sales_SalesOrderHeader")
northwind_orders_df = df_NorthWind.get("Orders")
aenc_sales_order_df = df_aenc.get("Sales_Order")

#  2. Voeg Source toe aan elke DataFrame 
sales_adventureworks_df["Source"] = "AdventureWorks"
northwind_orders_df["Source"] = "NorthWind"
aenc_sales_order_df["Source"] = "AENC"

#  3. Hernoem kolommen voor consistentie 
northwind_orders_df.rename(columns={
    "OrderID": "SalesOrderID",
    "OrderDate": "OrderDate",
    "ShippedDate": "ShipDate",
    "CustomerID": "CustomerID",
    "EmployeeID": "SalesPersonID",
    "Freight": "SubTotal",
    "ShipVia": "ShipVia",
    "ShipCountry": "ShipCountry",
    "ShipRegion": "ShipRegion",
    "ShipCity": "ShipCity"
}, inplace=True)

aenc_sales_order_df.rename(columns={
    "id": "SalesOrderID",
    "order_date": "OrderDate",
    "cust_id": "CustomerID",
    "sales_rep": "SalesPersonID",
    "region": "ShipRegion"
}, inplace=True)

#  4. Zorg dat CustomerID overal een string is 
sales_adventureworks_df["CustomerID"] = sales_adventureworks_df["CustomerID"].astype(str)
northwind_orders_df["CustomerID"] = northwind_orders_df["CustomerID"].astype(str)
aenc_sales_order_df["CustomerID"] = aenc_sales_order_df["CustomerID"].astype(str)

# Ook in de customer_mapping DataFrame
customer_mapping["CustomerID"] = customer_mapping["CustomerID"].astype(str)

#  5. Ontbrekende kolommen aanvullen met None 
for df in [sales_adventureworks_df, northwind_orders_df, aenc_sales_order_df]:
    for col in ["RevisionNumber", "TerritoryID", "SubTotal", "DueDate",
                "ShipDate", "ShipVia", "ShipCountry", "ShipRegion", "ShipCity"]:
        if col not in df.columns:
            df[col] = None

#  6. Selecteer en orden de relevante kolommen 
cols = [
    "SalesOrderID", "RevisionNumber", "OrderDate", "DueDate", "ShipDate", 
    "CustomerID", "SalesPersonID", "TerritoryID", "SubTotal", 
    "ShipVia", "ShipCountry", "ShipRegion", "ShipCity", "Source"
]

sales_adventureworks_df = sales_adventureworks_df[cols]
northwind_orders_df = northwind_orders_df[cols]
aenc_sales_order_df = aenc_sales_order_df[cols]

#  7. Concateneer de drie DataFrames 
merged_sales_salesOrderHeader = pd.concat(
    [sales_adventureworks_df, northwind_orders_df, aenc_sales_order_df], 
    ignore_index=True
)

#  8. CustomerID bijwerken via customer_mapping 
# Hier nemen we aan dat 'customer_mapping' al bestaat, met kolommen ["CustomerID", "Source", "MergedCustomerID"].
merged_sales_salesOrderHeader = merged_sales_salesOrderHeader.merge(
    customer_mapping,
    on=["CustomerID", "Source"],
    how="left"
)

# Vervang de CustomerID door MergedCustomerID
merged_sales_salesOrderHeader["CustomerID"] = merged_sales_salesOrderHeader["MergedCustomerID"]
merged_sales_salesOrderHeader.drop(columns=["MergedCustomerID"], inplace=True)

#  9. SalesPersonID bijwerken via employee_mapping 
# Hier nemen we aan dat 'employee_mapping' al bestaat, met kolommen ["BusinessEntityID", "Source", "MergedBusinessEntityID"].
merged_sales_salesOrderHeader = merged_sales_salesOrderHeader.merge(
    employee_mapping,
    left_on=["SalesPersonID", "Source"],
    right_on=["BusinessEntityID", "Source"],
    how="left",
    suffixes=("", "_emp")
)

# Vervang SalesPersonID door de gemapte MergedBusinessEntityID
merged_sales_salesOrderHeader["SalesPersonID"] = merged_sales_salesOrderHeader["MergedBusinessEntityID"]
merged_sales_salesOrderHeader.drop(columns=["BusinessEntityID", "MergedBusinessEntityID"], inplace=True)

#  10. Sla op in het data model en exporteer 
dfs_sourcedatamodel["Sales_SalesOrderHeader"] = merged_sales_salesOrderHeader
print(merged_sales_salesOrderHeader.head())




  SalesOrderID RevisionNumber          OrderDate            DueDate  \
0        43659              8  05/31/21 00:00:00  06/12/21 00:00:00   
1        43660              8  05/31/21 00:00:00  06/12/21 00:00:00   
2        43661              8  05/31/21 00:00:00  06/12/21 00:00:00   
3        43662              8  05/31/21 00:00:00  06/12/21 00:00:00   
4        43663              8  05/31/21 00:00:00  06/12/21 00:00:00   

            ShipDate  CustomerID SalesPersonID TerritoryID    SubTotal  \
0  06/07/21 00:00:00       19527           279           5  20565.6206   
1  06/07/21 00:00:00       19374           279           5   1294.2529   
2  06/07/21 00:00:00       19436           282           6  32726.4786   
3  06/07/21 00:00:00       19696           282           6  28832.5289   
4  06/07/21 00:00:00       19267           276           4    419.4589   

  ShipVia ShipCountry ShipRegion ShipCity          Source  
0    None        None       None     None  AdventureWorks  
1    Non

<h3> Table: Sales_SalesOrderDetail
<h4> Bronnen: Sales_SalesOrderDetail + OrderDetails + sales_order_item + product(aenc)

In [54]:
# 1. Laad de datasets 
Sales_SalesOrderDetail = df_AdventureWorks.get("Sales_SalesOrderDetail")
Sales_Order_Item = df_aenc.get("Sales_Order_Item")
OrderDetails = df_NorthWind.get("OrderDetails")
df_product2 = df_aenc.get("Product")  

df_product2.rename(columns={"id": "ProductID"}, inplace=True)

# 2. Voeg Source toe aan elke DataFrame 
Sales_SalesOrderDetail["Source"] = "AdventureWorks"
Sales_Order_Item["Source"] = "AENC"
OrderDetails["Source"] = "NorthWind"

# 3. Verwerk de AdventureWorks data 
Sales_SalesOrderDetail = Sales_SalesOrderDetail.copy()
Sales_SalesOrderDetail = Sales_SalesOrderDetail[[
    "SalesOrderID", "SalesOrderDetailID", "OrderQty", "ProductID", 
    "UnitPrice", "UnitPriceDiscount", "LineTotal", "Source"
]]

# 4. Verwerk de AENC data 
Sales_Order_Item = Sales_Order_Item.copy()
Sales_Order_Item.rename(columns={
    "id": "SalesOrderID",
    "line_id": "SalesOrderDetailID",
    "prod_id": "ProductID",
    "quantity": "OrderQty"
}, inplace=True)

if df_product2 is not None:
    Sales_Order_Item = Sales_Order_Item.merge(df_product2[["ProductID", "unit_price"]], on="ProductID", how="left")
    Sales_Order_Item.rename(columns={"unit_price": "UnitPrice"}, inplace=True)
else:
    print("The table does not exist in database.")

# 5. Verwerk de NorthWind data 
OrderDetails = OrderDetails.copy()
OrderDetails.rename(columns={
    "OrderID": "SalesOrderID",
    "Quantity": "OrderQty",
    "Discount": "UnitPriceDiscount"
}, inplace=True)

OrderDetails["SalesOrderDetailID"] = OrderDetails.groupby("SalesOrderID").cumcount() + 1
OrderDetails["LineTotal"] = None
OrderDetails = OrderDetails[[
    "SalesOrderID", "SalesOrderDetailID", "OrderQty", "ProductID", 
    "UnitPrice", "UnitPriceDiscount", "LineTotal", "Source"
]]

# 6. Combineer de drie bronnen 
merged_orderdetail = pd.concat([Sales_SalesOrderDetail, Sales_Order_Item, OrderDetails], ignore_index=True)

# 7. Werk de ProductID bij via product_mapping 
merged_orderdetail = merged_orderdetail.merge(
    product_mapping,
    on=["ProductID", "Source"],
    how="left"
)
merged_orderdetail["ProductID"] = merged_orderdetail["MergedID"]
merged_orderdetail.drop(columns=["MergedID"], inplace=True)

# 8. Verwijder de kolom ShipDate als die bestaat
if "ship_date" in merged_orderdetail.columns:
    merged_orderdetail = merged_orderdetail.drop(columns=["ship_date"])

# 9. Sla het resultaat op in het datamodel 
dfs_sourcedatamodel["Sales_SalesOrderDetail"] = merged_orderdetail

# 10. Print preview
print(merged_orderdetail.head())


  SalesOrderID SalesOrderDetailID OrderQty  ProductID UnitPrice  \
0        43659                  1        1        281  2024.994   
1        43659                  2        3        282  2024.994   
2        43659                  3        1        283  2024.994   
3        43659                  4        1        276  2039.994   
4        43659                  5        1        277  2039.994   

  UnitPriceDiscount LineTotal          Source  
0               0.0  2024.994  AdventureWorks  
1               0.0  6074.982  AdventureWorks  
2               0.0  2024.994  AdventureWorks  
3               0.0  2039.994  AdventureWorks  
4               0.0  2039.994  AdventureWorks  


<h3> Vullen SDM


In [55]:
for tabel in dfs_sourcedatamodel:
    print(f"{tabel}: {dfs_sourcedatamodel[tabel].shape}")

Production_ProductCategory: (12, 7)
Production_Product: (591, 34)
Purchasing_Vendor: (104, 8)
Sales_Store: (701, 5)
EmployeeTerritories: (49, 2)
Territories: (53, 3)
Region: (4, 2)
Bonus: (33, 3)
Sales_Customer: (20037, 10)
Sales_SalesTerritory: (10, 10)
Purchasing_PurchaseOrderHeader: (4012, 13)
Purchasing_PurchaseOrderDetail: (8845, 12)
Suppliers: (29, 12)
Person_Person: (20056, 12)
HumanResources_Department: (21, 5)
HumanResources_Employee: (20056, 10)
Person_Address: (19614, 7)
Shippers: (3, 3)
Sales_SalesOrderHeader: (32945, 14)
Sales_SalesOrderDetail: (124575, 8)


<h3> Connectie leggen SDM 

In [56]:
# Verbindingsgegevens
server = '127.0.0.1'        
port = '1433'               
database2 = 'SDMProject'         
username = 'SA'             
password = 'iDTyjZx7dRL4'  

# Connection string
connection_string2 = (
    f"DRIVER={{ODBC Driver 17 for SQL Server}};"
    f"SERVER={server},{port};"
    f"DATABASE={database2};"
    f"UID={username};"
    f"PWD={password};"
    "TrustServerCertificate=yes;"
    "Timeout=30;"
)

# Maak verbinding met de database
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

# Haal alle tabellen op``
cursor.execute("SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE = 'BASE TABLE'")
tables = [row.TABLE_NAME for row in cursor.fetchall()]


# Sluit de verbinding
conn.close()





<h3> Converteer datatypen met datums naar DATETIME

In [57]:

def convert_all_dates(dw):
    date_keywords = ["date", "time"]  # Zoek naar deze woorden in kolomnamen
    
    for table_name, df in dw.items():
        for col in df.columns:
            if any(keyword in col.lower() for keyword in date_keywords):  # Check op 'date' of 'time'
                try:
                    df[col] = pd.to_datetime(df[col], errors='coerce')  # Converteer naar datetime
                    print(f"✅ Geconverteerd: {table_name}.{col}")
                except Exception as e:
                    print(f"⚠️ Fout bij converteren van {col} in {table_name}: {e}")
    
    return dw

# Pas de functie toe op je DataFrame-collectie
dfs_sourcedatamodel = convert_all_dates(dfs_sourcedatamodel)


✅ Geconverteerd: Production_ProductCategory.ModifiedDate
✅ Geconverteerd: Production_Product.SellStartDate
✅ Geconverteerd: Production_Product.SellEndDate
✅ Geconverteerd: Production_Product.ModifiedDate
✅ Geconverteerd: Purchasing_Vendor.ModifiedDate
✅ Geconverteerd: Sales_Store.ModifiedDate
✅ Geconverteerd: Bonus.bonus_date
✅ Geconverteerd: Sales_Customer.ModifiedDate
✅ Geconverteerd: Sales_SalesTerritory.ModifiedDate
✅ Geconverteerd: Purchasing_PurchaseOrderHeader.OrderDate
✅ Geconverteerd: Purchasing_PurchaseOrderHeader.ShipDate
✅ Geconverteerd: Purchasing_PurchaseOrderHeader.ModifiedDate
✅ Geconverteerd: Purchasing_PurchaseOrderDetail.DueDate
✅ Geconverteerd: Purchasing_PurchaseOrderDetail.ModifiedDate
✅ Geconverteerd: Person_Person.ModifiedDate
✅ Geconverteerd: HumanResources_Department.ModifiedDate
✅ Geconverteerd: HumanResources_Employee.BirthDate
✅ Geconverteerd: HumanResources_Employee.HireDate
✅ Geconverteerd: Sales_SalesOrderHeader.OrderDate
✅ Geconverteerd: Sales_SalesOrde

<h3> Vullen van database

In [58]:
def clean_nan_values(dw):
    for table_name, df in dw.items():
        # Zet alle NaN naar None zodat SQL Server NULL kan verwerken
        dw[table_name] = df.astype(object).where(pd.notnull(df), None)
    return dw

# Pas toe op je dataWarehouse dictionary
dfs_sourcedatamodel = clean_nan_values(dfs_sourcedatamodel)


def upload_dataframes_to_sql(dw):
    try:
        with pyodbc.connect(connection_string2, autocommit=False) as conn:
            cursor = conn.cursor()
            cursor.fast_executemany = True  # Maakt batch-inserts sneller
            
            print("⏳ Uitschakelen van FOREIGN KEY constraints...")
            cursor.execute("EXEC sp_MSforeachtable 'ALTER TABLE ? NOCHECK CONSTRAINT ALL'")
            conn.commit()
            
            # Loop over elke tabel
            for table_name, df in dw.items():
                print(f"\nBezig met uploaden van tabel: {table_name}...")
                
                columns = ', '.join([f'[{col}]' for col in df.columns])
                placeholders = ', '.join(['?'] * len(df.columns))
                insert_query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"
                
                # Gebruik batch commit
                batch_size = 1000
                data_list = [tuple(row) for _, row in df.iterrows()]
                
                for i in range(0, len(data_list), batch_size):
                    try:
                        cursor.executemany(insert_query, data_list[i:i+batch_size])
                        conn.commit()
                        print(f"  ✅ Batch {i//batch_size + 1} geüpload ({len(data_list[i:i+batch_size])} rijen)")
                    except pyodbc.Error as e:
                        conn.rollback()
                        print(f"  ❌ Fout in batch {i//batch_size + 1}: {str(e)}")
                
            print("\n⏳ Herinschakelen van FOREIGN KEY constraints...")
            cursor.execute("EXEC sp_MSforeachtable 'ALTER TABLE ? CHECK CONSTRAINT ALL'")
            conn.commit()
            
            print("\n🎉 Upload voltooid voor alle tabellen!")
            cursor.close()
            
    except pyodbc.Error as e:
        print(f"❌ Databasefout: {e}")

upload_dataframes_to_sql(dfs_sourcedatamodel)




⏳ Uitschakelen van FOREIGN KEY constraints...

Bezig met uploaden van tabel: Production_ProductCategory...
  ✅ Batch 1 geüpload (12 rijen)

Bezig met uploaden van tabel: Production_Product...
  ✅ Batch 1 geüpload (591 rijen)

Bezig met uploaden van tabel: Purchasing_Vendor...
  ✅ Batch 1 geüpload (104 rijen)

Bezig met uploaden van tabel: Sales_Store...
  ✅ Batch 1 geüpload (701 rijen)

Bezig met uploaden van tabel: EmployeeTerritories...
  ✅ Batch 1 geüpload (49 rijen)

Bezig met uploaden van tabel: Territories...
  ✅ Batch 1 geüpload (53 rijen)

Bezig met uploaden van tabel: Region...
  ✅ Batch 1 geüpload (4 rijen)

Bezig met uploaden van tabel: Bonus...
  ✅ Batch 1 geüpload (33 rijen)

Bezig met uploaden van tabel: Sales_Customer...
  ✅ Batch 1 geüpload (1000 rijen)
  ✅ Batch 2 geüpload (1000 rijen)
  ✅ Batch 3 geüpload (1000 rijen)
  ✅ Batch 4 geüpload (1000 rijen)
  ✅ Batch 5 geüpload (1000 rijen)
  ✅ Batch 6 geüpload (1000 rijen)
  ✅ Batch 7 geüpload (1000 rijen)
  ✅ Batch 8 geüp